In [29]:
import math
import numpy as np

In [30]:
class BaseWindow():
    """
    Class to define a basic windows
    Example steps of defining a window
    ----------
    (1) window = BaseWindow()
    ----------
    Necessary Parameters:
    (1) height: window height in mm
    (2) width: window width in mm
    (3) glazing: total glas thickness in mm
    (4) opening_direction:  "inward", "outward"
    (5) window_type: type of window 1: roof, 2: bottom-hung, 3: top-hung, 4: side-hung
    """
    
    def __init__(
        self,
        height: int, 
        width: int, 
        glazing: float, 
        opening_direction: str,
        window_type: int,
        wind_load: int = 0, 
        hinge_position: tuple = (10,10)
    ):
        # geometry and glas
        self.height = height
        self.width = width
        self.surface = self.height*self.width/1000000
        
        # load
        self.glazing = glazing
        self.sash_weight = self.surface*(self.glazing/1000)*2500
        self.filling_weight = self.sash_weight/self.surface if self.surface != 0 else 0
        
        # opening direction
        self.direction = opening_direction
        
        # window type
        self.type = window_type
        
        # environmental influences
        self.wind_load_N = wind_load
        
        # hinge position
        self.hinge_position = hinge_position
        
    def set_wind_load(self, value, load_type='speed'):
        
        if load_type == 'speed':
            self.wind_speed = value
            self.wind_pressure = 0.5 * 1.25 * value**2
            self.wind_load_N = self.wind_pressure * self.surface
        elif load_type == 'pressure':
            self.wind_pressure = value
            self.wind_load_N = self.wind_pressure * self.surface
            self.wind_speed = self.wind_pressure/(0.5 * 1.25)
        else:
            raise ValueError("Unexpected load type please select either 'speed' (default) or 'pressure'")
    
    def load_moment(
        self,
        opening_angle=None,
        i_dim=57,
        inclination=90,
        max_angle=60
    ):
        if opening_angle is None:
            opening_angle = np.linspace(
                0, 
                max_angle,
                max_angle + 1
            )
            
        angle_rad = np.radians(opening_angle)
        weight_force = self.sash_weight * 9.81
        self.inclination = inclination
        
        if self.direction == "outward":
            self.wind_load_N = self.wind_load_N*(-1)
        
        if self.type == 2:
            self.inclination = self.inclination*(-1)
        
        inclination_rad = np.radians(self.inclination)
        
        inclination_red = {
        "sin": np.sin(inclination_rad),
        "cos": np.cos(inclination_rad)
        }
        
        centre_shift = {
        "sin": i_dim * np.sin(angle_rad),
        "cos": i_dim * np.cos(angle_rad)
        }
        
        lever_arm = {
            "sin": (
                self.height / 2 + self.hinge_position[1]
            )*np.sin(angle_rad),
            
            "cos": (
                -self.height / 2 + self.hinge_position[1]
            )*np.cos(angle_rad)
        }
        
        # force component in x direction
        load_comp_x = weight_force * inclination_red["sin"]\
                    *(lever_arm["sin"]-centre_shift["cos"])
        
        # force component in y direction
        load_comp_y = weight_force * inclination_red["cos"]\
                    *(lever_arm["cos"]-centre_shift["sin"])
        
        load_moment = (load_comp_x + load_comp_y) / 1000
        
        if self.wind_load_N:
            if (180.0001-(self.inclination % 360)) < 0:
                self.wind_load_N = self.wind_load_N*(-1) 
            
            load_moment = load_moment + self.wind_load_N * self.height / 2000 *\
                          abs(np.sin(inclination_rad+angle_rad))*math.sin(inclination_rad+angle_rad)
        
        load = dict(enumerate(np.round(load_moment.flatten(),2), 0))
        
        return load

In [31]:
window = BaseWindow(
    height = 1000, 
    width = 1000, 
    glazing = 12, 
    opening_direction = "inward",
    window_type = 1,
)

In [32]:
window.sash_weight

30.0

In [34]:
window.load_moment()

{0: -16.78,
 1: -14.15,
 2: -11.53,
 3: -8.9,
 4: -6.26,
 5: -3.63,
 6: -0.99,
 7: 1.64,
 8: 4.28,
 9: 6.91,
 10: 9.54,
 11: 12.17,
 12: 14.8,
 13: 17.42,
 14: 20.03,
 15: 22.64,
 16: 25.25,
 17: 27.84,
 18: 30.43,
 19: 33.0,
 20: 35.57,
 21: 38.13,
 22: 40.67,
 23: 43.2,
 24: 45.72,
 25: 48.23,
 26: 50.72,
 27: 53.19,
 28: 55.65,
 29: 58.09,
 30: 60.52,
 31: 62.92,
 32: 65.31,
 33: 67.68,
 34: 70.02,
 35: 72.35,
 36: 74.65,
 37: 76.93,
 38: 79.19,
 39: 81.42,
 40: 83.63,
 41: 85.81,
 42: 87.97,
 43: 90.09,
 44: 92.2,
 45: 94.27,
 46: 96.31,
 47: 98.33,
 48: 100.32,
 49: 102.27,
 50: 104.2,
 51: 106.09,
 52: 107.95,
 53: 109.77,
 54: 111.57,
 55: 113.33,
 56: 115.05,
 57: 116.74,
 58: 118.4,
 59: 120.01,
 60: 121.6}